In [1]:
import pandas as pd
import numpy as np
import json
from geopy.distance import geodesic
from searoute import searoute
import folium
from shapely.geometry import Point, Polygon
from geographiclib.geodesic import Geodesic
import random

Port Information + Warzone Area

In [14]:
PORTS = {
    # ----- EXPORT PORTS -----
    "Rosario": {"country": "Argentina", "role": "export", "coordinates": (-32.96, -60.76), "cargo_types": ["HSS", "maize"]},
    "Kwinana": {"country": "Australia", "role": "export", "coordinates": (-32.45, 115.65), "cargo_types": ["HSS", "barley"]},
    "Itaqui": {"country": "Brazil", "role": "export", "coordinates": (-28.77, -52.28), "cargo_types": ["HSS", "maize"]},
    "Santos": {"country": "Brazil", "role": "both", "coordinates": (-23.97, -46.33), "cargo_types": ["HSS", "maize"]},  # Export + Import
    "Baie-Comeau": {"country": "Canada", "role": "export", "coordinates": (49.45, -68.25), "cargo_types": ["HSS"]},
    "Rouen": {"country": "France", "role": "export", "coordinates": (49.44, 1.09), "cargo_types": ["HSS", "maize", "barley"]},
    "Hamburg": {"country": "Germany", "role": "export", "coordinates": (53.55, 9.99), "cargo_types": ["HSS"]},
    "Novorossiysk": {"country": "Russia", "role": "export", "coordinates": (44.73, 37.80), "cargo_types": ["HSS", "barley"]},
    "Yuzhny": {"country": "Ukraine", "role": "export", "coordinates": (46.65, 30.55), "cargo_types": ["HSS", "maize", "barley"]},
    "Tilbury": {"country": "United Kingdom", "role": "export", "coordinates": (51.42, 0.20), "cargo_types": ["HSS", "barley"]},
    "New Orleans": {"country": "United States", "role": "export", "coordinates": (29.95, -90.07), "cargo_types": ["HSS", "maize"]},
    "Tacoma": {"country": "United States", "role": "export", "coordinates": (47.25, -122.45), "cargo_types": ["HSS", "maize", "barley"]},
    "Constanta": {"country": "Romania", "role": "export", "coordinates": (44.15, 28.63), "cargo_types": ["HSS", "maize"]},
    "Bangkok": {"country": "Thailand", "role": "export", "coordinates": (13.75, 100.50), "cargo_types": ["barley"]},

    # ----- IMPORT PORTS -----
    "Bejaia": {"country": "Algeria", "role": "import", "coordinates": (36.75, 5.05), "cargo_types": ["HSS", "maize", "barley"]},
    "Luanda": {"country": "Angola", "role": "import", "coordinates": (-8.83, 13.25), "cargo_types": ["HSS"]},
    "Chittagong": {"country": "Bangladesh", "role": "import", "coordinates": (22.34, 91.82), "cargo_types": ["HSS", "maize"]},
    "Cotonou": {"country": "Benin", "role": "import", "coordinates": (6.37, 2.42), "cargo_types": ["barley"]},
    "Douala": {"country": "Cameroon", "role": "import", "coordinates": (4.05, 9.70), "cargo_types": ["barley"]},
    "San Vicente": {"country": "Chile", "role": "import", "coordinates": (-37.22, -73.61), "cargo_types": ["HSS", "maize"]},
    "Dalian": {"country": "China", "role": "import", "coordinates": (38.92, 121.62), "cargo_types": ["HSS", "maize", "barley"]},
    "Kaohsiung": {"country": "Chinese Taipei", "role": "import", "coordinates": (22.63, 120.30), "cargo_types": ["HSS", "maize"]},
    "Barranquilla": {"country": "Colombia", "role": "import", "coordinates": (10.96, -74.80), "cargo_types": ["HSS", "maize"]},
    "Abidjan": {"country": "Côte d'Ivoire", "role": "import", "coordinates": (5.32, -4.03), "cargo_types": ["barley"]},
    "Djibouti": {"country": "Djibouti", "role": "import", "coordinates": (11.59, 43.15), "cargo_types": ["HSS", "barley"]},
    "Alexandria": {"country": "Egypt", "role": "import", "coordinates": (31.20, 29.92), "cargo_types": ["HSS"]},
    "Tema": {"country": "Ghana", "role": "import", "coordinates": (5.66, -0.02), "cargo_types": ["barley"]},
    "Conakry": {"country": "Guinea", "role": "import", "coordinates": (9.51, -13.71), "cargo_types": ["barley"]},
    "Bissau": {"country": "Guinea-Bissau", "role": "import", "coordinates": (11.85, -15.56), "cargo_types": ["barley"]},
    "Jakarta": {"country": "Indonesia", "role": "import", "coordinates": (-6.12, 106.84), "cargo_types": ["HSS", "maize"]},
    "Bandar Abbas": {"country": "Iran", "role": "import", "coordinates": (27.18, 56.27), "cargo_types": ["HSS", "maize", "barley"]},
    "Basrah": {"country": "Iraq", "role": "import", "coordinates": (30.50, 47.78), "cargo_types": ["HSS", "barley"]},
    "Genoa": {"country": "Italy", "role": "import", "coordinates": (44.41, 8.94), "cargo_types": ["HSS", "maize", "barley"]},
    "Yokohama": {"country": "Japan", "role": "import", "coordinates": (35.45, 139.64), "cargo_types": ["HSS", "maize", "barley"]},
    "Aqaba": {"country": "Jordan", "role": "import", "coordinates": (29.53, 35.01), "cargo_types": ["barley"]},
    "Mombasa": {"country": "Kenya", "role": "import", "coordinates": (-4.05, 39.67), "cargo_types": ["barley"]},
    "Inchon": {"country": "Korea", "role": "import", "coordinates": (37.45, 126.62), "cargo_types": ["HSS", "maize"]},
    "Monrovia": {"country": "Liberia", "role": "import", "coordinates": (6.31, -10.80), "cargo_types": ["barley"]},
    "Tripoli": {"country": "Libya", "role": "import", "coordinates": (32.90, 13.18), "cargo_types": ["HSS", "barley"]},
    "Toamasina": {"country": "Madagascar", "role": "import", "coordinates": (-18.15, 49.38), "cargo_types": ["barley"]},
    "Pasir Gudang": {"country": "Malaysia", "role": "import", "coordinates": (1.48, 103.91), "cargo_types": ["HSS", "maize"]},
    "Veracruz": {"country": "Mexico", "role": "import", "coordinates": (19.18, -96.14), "cargo_types": ["HSS", "maize"]},
    "Casablanca": {"country": "Morocco", "role": "import", "coordinates": (33.57, -7.59), "cargo_types": ["HSS"]},
    "Maputo": {"country": "Mozambique", "role": "import", "coordinates": (-25.97, 32.58), "cargo_types": ["barley"]},
    "Rotterdam": {"country": "Netherlands", "role": "import", "coordinates": (51.92, 4.48), "cargo_types": ["HSS", "maize", "barley"]},
    "Lagos": {"country": "Nigeria", "role": "import", "coordinates": (6.52, 3.37), "cargo_types": ["barley"]},
    "Port Sultan Qaboos": {"country": "Oman", "role": "import", "coordinates": (23.62, 58.62), "cargo_types": ["HSS", "maize", "barley"]},
    "Salalah": {"country": "Oman", "role": "import", "coordinates": (17.05, 54.12), "cargo_types": ["HSS", "maize", "barley"]},
    "Karachi": {"country": "Pakistan", "role": "import", "coordinates": (24.86, 67.01), "cargo_types": ["barley"]},
    "Gdynia": {"country": "Poland", "role": "import", "coordinates": (54.52, 18.54), "cargo_types": ["HSS", "maize"]},
    "Baltiysk": {"country": "Russia", "role": "import", "coordinates": (54.64, 20.04), "cargo_types": ["HSS"]},
    "Jeddah": {"country": "Saudi Arabia", "role": "import", "coordinates": (21.53, 39.19), "cargo_types": ["barley"]},
    "Dakar": {"country": "Senegal", "role": "import", "coordinates": (14.69, -17.44), "cargo_types": ["barley"]},
    "Mogadishu": {"country": "Somalia", "role": "import", "coordinates": (2.04, 45.34), "cargo_types": ["barley"]},
    "Durban": {"country": "South Africa", "role": "import", "coordinates": (-29.86, 31.02), "cargo_types": ["HSS", "maize", "barley"]},
    "Cartagena": {"country": "Spain", "role": "import", "coordinates": (37.55, -0.99), "cargo_types": ["HSS", "barley"]},
    "Colombo": {"country": "Sri Lanka", "role": "import", "coordinates": (6.93, 79.85), "cargo_types": ["barley"]},
    "Port Sudan": {"country": "Sudan", "role": "import", "coordinates": (19.62, 37.28), "cargo_types": ["HSS", "barley"]},
    "Dar es Salaam": {"country": "Tanzania", "role": "import", "coordinates": (-6.80, 39.27), "cargo_types": ["barley"]},
    "Koh Sichang": {"country": "Thailand", "role": "import", "coordinates": (13.18, 100.85), "cargo_types": ["barley"]},
    "Lome": {"country": "Togo", "role": "import", "coordinates": (6.13, 1.22), "cargo_types": ["barley"]},
    "Bizerte": {"country": "Tunisia", "role": "import", "coordinates": (37.27, 9.87), "cargo_types": ["HSS"]},
    "Canakkale": {"country": "Turkey", "role": "import", "coordinates": (40.15, 26.41), "cargo_types": ["HSS", "maize"]},
    "Liverpool": {"country": "United Kingdom", "role": "import", "coordinates": (53.41, -2.98), "cargo_types": ["HSS", "maize", "barley"]},
    "Puerto Cabello": {"country": "Venezuela", "role": "import", "coordinates": (10.47, -68.00), "cargo_types": ["HSS", "maize"]},
    "Ho Chi Minh City": {"country": "Vietnam", "role": "import", "coordinates": (10.77, 106.67), "cargo_types": ["HSS", "maize"]},
    "Saleef": {"country": "Yemen", "role": "import", "coordinates": (15.36, 48.86), "cargo_types": ["HSS", "barley"]},
    'Cape Town': {'coordinates': (-33.92, 18.42), 'role': 'detour', 'cargo_types': []},
}

# --- CARGO-SPECIFIC HUB LISTS (from OECD Table 5 & Annex C) ---
TOP_EXPORT_HUBS = {
    'maize': {'Santos', 'New Orleans', 'Yuzhny'},
    'wheat': {'Baie-Comeau', 'New Orleans', 'Novorossiysk'},
    'barley': {'Kwinana', 'Tilbury', 'Novorossiysk'}
}

CARGO_TO_IMPORT_HUBS = {
    # HSS (Heavy Grains: Wheat, Durum, Sorghum, Soybeans)
    'HSS': {'Cartagena', 'Alexandria', 'Turkey', 'Morocco'},
    
    # Maize
    'maize': {'Rotterdam', 'Genoa', 'Alexandria', 'Algeria'},
    
    # Barley
    'barley': {'Jeddah', 'Rotterdam', 'Genoa', 'Cartagena'}
}

WAR_ZONES = {
    'Gulf of Aden & Southern Red Sea': {
        'bounds': [(18.0, 38.0), (16.6417, 53.1083), (14.9167, 53.8333), (6.75, 48.75), (1.6667, 41.5667), (6.75, 48.75), (18.0, 38.0)],
        'description': 'High risk of piracy. Critical for routes from India/China to Europe via Suez.'
    },
    'Sea of Azov': {
        'bounds': [(45.1809, 29.7655), (45.1898, 29.8523), (45.1911, 29.9927), (45.0892, 30.0401), (44.7771, 30.9787), (43.3854, 40.0100), (45.1809, 29.7655)],
        'description': 'High risk due to Ukraine conflict.'
    },
    'Black Sea (Western Maritime)': {
        'bounds': [(45.1809, 29.7655), (44.0480, 31.4100), (43.0, 42.0), (40.0, 42.0), (40.0, 30.0), (45.0, 30.0), (45.1809, 29.7655)],
        'description': 'Western part of Black Sea near Ukraine/Russia. Avoid unless essential.'
    },
    'Gulf of Guinea': {
        'bounds': [(6.1125, 1.2), (0.6667, 3.0), (0.6667, 8.7), (6.1125, 8.7), (6.1125, 1.2)],
        'description': 'High risk of piracy and armed robbery.'
    }
}

key_hubs = {"Rotterdam", "Alexandria", "Cartagena", "Genoa", "Jeddah"}

Port Parameters (To be changed)

In [11]:
# Port charges (USD)
HUB_PORT_CHARGE = 295_000    # midpoint of 280k-310k
NONHUB_PORT_CHARGE = 400_000 # midpoint of 350k-450k

# Demurrage expected (USD total expectation)
HUB_DEMURRAGE = 50_000      # midpoint of 37.5k-62.5k
NONHUB_DEMURRAGE = 137_500  # midpoint of 100k-175k

# Laytime planned (days per port)
HUB_LAYTIME_DAYS = 1.75
NONHUB_LAYTIME_DAYS = 4.0

# War Risk Insurance (USD/day)
HUB_WRI_PER_DAY = 1_000      # small baseline
NONHUB_WRI_PER_DAY = 7_500   # midpoint of 0-15k

# Other unchanged parameters
TCH_per_day = 22_000
fuel_consumption_per_day = 28
fuel_price_per_mt = 640
AVERAGE_SPEED_KNOTS = 14
KNOT_TO_KMH = 1.852

Function helpers

In [16]:
def create_polygon_from_coords(coords):
    """Convert lat/lon coordinates to Shapely Polygon."""
    return Polygon([(lon, lat) for lat, lon in coords])

def point_in_any_war_zone(lat, lon):
    """Check if point is inside any defined war zone."""
    point = Point(lon, lat)
    for zone_name, zone_info in WAR_ZONES.items():
        poly = create_polygon_from_coords(zone_info['bounds'])
        if poly.contains(point):
            return True, zone_name
    return False, None

def get_sea_route_points(origin_lon, origin_lat, dest_lon, dest_lat):
    """Return sea route coordinates from searoute. Always returns a list, even if route fails."""
    try:
        route = searoute([origin_lon, origin_lat], [dest_lon, dest_lat])
        coords = route.get('geometry', {}).get('coordinates', [])
        if coords and len(coords) > 1:
            return coords
        else:
            return []  # return empty list instead of None
    except Exception as e:
        print(f"Error calculating route: {e}")
        return []  # never return None


def route_crosses_war_zone(origin_coords, dest_coords, num_waypoints=100):
    """Generate waypoints along the great circle path and check if any fall in a war zone."""
    lat1, lon1 = origin_coords
    lat2, lon2 = dest_coords
    g = Geodesic.WGS84.Inverse(lat1, lon1, lat2, lon2)
    total_distance = g['s12']  # meters
    for i in range(num_waypoints + 1):
        fraction = i / num_waypoints
        g_line = Geodesic.WGS84.Direct(lat1, lon1, g['azi1'], fraction * total_distance)
        waypoint_lat = g_line['lat2']
        waypoint_lon = g_line['lon2']
        crosses, zone_name = point_in_any_war_zone(waypoint_lat, waypoint_lon)
        if crosses:
            return True, zone_name
    return False, None

def searoute_distance_km(route_coords):
    """Calculate total distance (km) along a searoute coordinate list."""
    from geopy.distance import geodesic

    if not route_coords or len(route_coords) < 2:
        return None
    total_distance = 0
    for i in range(len(route_coords) - 1):
        # Searoute coords are [lon, lat]
        lon1, lat1 = route_coords[i]
        lon2, lat2 = route_coords[i + 1]
        total_distance += geodesic((lat1, lon1), (lat2, lon2)).km
    return total_distance

def route_waypoints(origin_coords, dest_coords, detour_cape=False):
    """Generate waypoints along straight line (or via Cape Town if detour)."""
    waypoints = []
    if detour_cape:
        # Cape Town coordinates
        cape_coords = (-33.92, 18.42)
        segments = [origin_coords, cape_coords, dest_coords]
    else:
        segments = [origin_coords, dest_coords]
    for i in range(len(segments)-1):
            lat1, lon1 = segments[i]
            lat2, lon2 = segments[i+1]
            for f in range(11):
                frac = f / 10
                lat = lat1 + (lat2 - lat1) * frac
                lon = lon1 + (lon2 - lon1) * frac
                waypoints.append((lat, lon))
    return waypoints

def create_ports_warzones_map():
    # Initialize map
    m = folium.Map(location=[0, 20], zoom_start=3, tiles='OpenStreetMap')

    # --- Add ports ---
    port_colors = {'export': 'green', 'import': 'blue', 'both': 'black'}
    for port_name, info in PORTS.items():
        lat, lon = info['coordinates']
        role = info['role']
        color = port_colors.get(role, 'gray')
        popup_text = f"{port_name}<br>Role: {role}"
        folium.Marker(
            location=[lat, lon],
            popup=popup_text,
            icon=folium.Icon(color=color, icon='ship', prefix='fa')
        ).add_to(m)

    # --- Add war zones ---
    colors = {'Gulf of Aden & Southern Red Sea':'red', 'Sea of Azov':'darkred', 'Black Sea (Western Maritime)':'orange', 'Gulf of Guinea':'purple'}
    for zone_name, zone_info in WAR_ZONES.items():
        poly_coords = [(lat, lon) for lat, lon in zone_info['bounds']]
        folium.Polygon(
            locations=poly_coords,
            color=colors.get(zone_name,'gray'),
            weight=3,
            fill=True,
            fillColor=colors.get(zone_name,'gray'),
            fillOpacity=0.2,
            popup=f"{zone_name}: {zone_info['description']}"
        ).add_to(m)

    # Save map
    m.save("ports_and_warzones_map.html")
    print("Map saved as 'ports_and_warzones_map.html'")

if __name__ == "__main__":
    create_ports_warzones_map()

def calculate_routes(port_pairs):
    """
    Calculate routes for given port pairs.
    Includes direct routes and optional top import hub routes based on cargo types.
    Returns a dictionary with route info.
    """
    routes_info = {}
    
    for origin, dest in port_pairs:
        if origin == dest:
            continue  # skip self-to-self
        
        origin_coords = PORTS[origin]['coordinates']
        dest_coords = PORTS[dest]['coordinates']
        
        # --- Direct route ---
        crosses_war_zone, crossed_zone = route_crosses_war_zone(origin_coords, dest_coords)
        sea_points = get_sea_route_points(origin_coords[1], origin_coords[0],
                                          dest_coords[1], dest_coords[0])
        if not sea_points:
            sea_points = []  # safety
        
        if not crosses_war_zone:
            distance = sum(geodesic((lat1, lon1), (lat2, lon2)).km
                           for (lon1, lat1), (lon2, lat2) in zip(sea_points[:-1], sea_points[1:]))
            routes_info[f"{origin} -> {dest}"] = {
                "waypoints": sea_points,
                "distance_km": distance,
                "war_zone_crossed": None,
                "High risk": "NIL"
            }
        else:
            # Detour via Cape Town
            cape_coords = PORTS['Cape Town']['coordinates']
            sea_points_1 = get_sea_route_points(origin_coords[1], origin_coords[0],
                                                cape_coords[1], cape_coords[0])
            sea_points_2 = get_sea_route_points(cape_coords[1], cape_coords[0],
                                                dest_coords[1], dest_coords[0])
            full_points = sea_points_1 + sea_points_2
            distance = sum(geodesic((lat1, lon1), (lat2, lon2)).km
                           for (lon1, lat1), (lon2, lat2) in zip(full_points[:-1], full_points[1:]))
            # Check if still intersects a war zone
            still_crosses, zone_name = route_crosses_war_zone(origin_coords, dest_coords)
            high_risk_label = zone_name if still_crosses else "NIL"
            
            routes_info[f"{origin} -> {dest}"] = {
                "waypoints": full_points,
                "distance_km": distance,
                "war_zone_crossed": crossed_zone,
                "High risk": high_risk_label
            }
        
        # --- Alternative top import hub routes based on destination cargo ---
        dest_cargo_types = PORTS[dest].get('cargo_types', [])
        for cargo in dest_cargo_types:
            top_hubs = CARGO_TO_IMPORT_HUBS.get(cargo, set())
            for hub_name in top_hubs:
                if hub_name == origin or hub_name == dest:
                    continue  # skip self
                if hub_name not in PORTS:
                    continue  # skip if hub not in PORTS dict
                
                hub_info = PORTS[hub_name]
                hub_coords = hub_info['coordinates']
                
                # Route: origin -> hub -> dest
                hub_sea_points_1 = get_sea_route_points(origin_coords[1], origin_coords[0],
                                                        hub_coords[1], hub_coords[0])
                hub_sea_points_2 = get_sea_route_points(hub_coords[1], hub_coords[0],
                                                        dest_coords[1], dest_coords[0])
                full_hub_points = hub_sea_points_1 + hub_sea_points_2
                distance_hub = sum(geodesic((lat1, lon1), (lat2, lon2)).km
                                   for (lon1, lat1), (lon2, lat2) in zip(full_hub_points[:-1], full_hub_points[1:]))
                
                # Check if still intersects a war zone
                still_crosses, zone_name = route_crosses_war_zone(origin_coords, dest_coords)
                high_risk_label = zone_name if still_crosses else "NIL"
                
                route_name = f"{origin} -> {hub_name} -> {dest}"
                routes_info[route_name] = {
                    "waypoints": full_hub_points,
                    "distance_km": distance_hub,
                    "war_zone_crossed": crossed_zone,
                    "High risk": high_risk_label
                }
    
    return routes_info

def visualize_routes_for_ports(port_pairs, map_filename="random_routes_map.html", routes_json="all_routes.json"):
    # Load the saved routes from JSON
    with open("all_routes_info.json", "r") as f:
        all_routes_info = json.load(f)
    
    # Initialize map
    m = folium.Map(location=[0, 20], zoom_start=3, tiles='OpenStreetMap')

    # --- Add ports ---
    port_colors = {'export': 'green', 'import': 'blue', 'both': 'black', 'detour': 'orange'}
    for port_name, info in PORTS.items():
        lat, lon = info['coordinates']
        role = info['role']
        color = port_colors.get(role, 'gray')
        folium.Marker(
            location=[lat, lon],
            popup=f"{port_name}<br>{role}",
            icon=folium.Icon(color=color, icon='ship', prefix='fa')
        ).add_to(m)

    # --- Add war zones ---
    zone_colors = {'Gulf of Aden & Southern Red Sea':'red', 'Sea of Azov':'darkred', 
                   'Black Sea (Western Maritime)':'orange', 'Gulf of Guinea':'purple'}
    for zone_name, zone_info in WAR_ZONES.items():
        poly_coords = [(lat, lon) for lat, lon in zone_info['bounds']]
        folium.Polygon(
            locations=poly_coords,
            color=zone_colors.get(zone_name,'gray'),
            weight=3,
            fill=True,
            fillColor=zone_colors.get(zone_name,'gray'),
            fillOpacity=0.2,
            popup=f"{zone_name}: {zone_info['description']}"
        ).add_to(m)

    # --- Plot routes for selected port pairs ---
    for origin, dest in port_pairs:
        for route_name, route_data in all_routes_info.items():
            # Check if the route belongs to this port pair
            if route_name.startswith(f"{origin} -> {dest}") or f"{origin} ->" in route_name:
                waypoints = route_data['waypoints']
                if len(waypoints) < 2:
                    continue  # skip empty routes
                
                # Color code
                if route_data['High risk'] != "NIL":
                    color = 'red'
                elif "->" in route_name.replace(f"{origin} -> {dest}", ""):
                    color = 'green'  # hub-based alternative
                else:
                    color = 'blue'   # direct route
                
                folium.PolyLine(
                    locations=[(lat, lon) for lon, lat in waypoints],
                    color=color,
                    weight=2,
                    opacity=0.7,
                    popup=f"{route_name}\nDistance: {route_data['distance_km']:.1f} km\nHigh risk: {route_data['High risk']}"
                ).add_to(m)

    # Save map
    m.save(map_filename)
    print(f"Map saved as '{map_filename}'")


def get_routes_from_json(origin, destination, routes_json="all_routes_info.json"):
    """
    Returns all available routes from JSON for a given port pair.
    """
    with open(routes_json, "r") as f:
        all_routes = json.load(f)

    filtered_routes = {}
    for route_name, route_info in all_routes.items():
        ports_in_route = route_name.split(" -> ")
        if ports_in_route[0] == origin and ports_in_route[-1] == destination:
            filtered_routes[route_name] = route_info

    return filtered_routes

def km_to_days(distance_km, speed_knots=AVERAGE_SPEED_KNOTS):
    speed_kmh = speed_knots * KNOT_TO_KMH
    hours = distance_km / speed_kmh
    return hours / 24.0

def parse_route_ports(route_name):
    parts = [p.strip() for p in route_name.split("->") if p.strip()]
    return parts

# --------------------------
# New cost function: per-port classification (hub/nonhub)
# --------------------------
def calculate_cost_with_port_types(route_name, route_info, key_stops=set(),
                                   hub_port_charge=HUB_PORT_CHARGE,
                                   nonhub_port_charge=NONHUB_PORT_CHARGE,
                                   hub_demurrage=HUB_DEMURRAGE,
                                   nonhub_demurrage=NONHUB_DEMURRAGE,
                                   hub_laytime=HUB_LAYTIME_DAYS,
                                   nonhub_laytime=NONHUB_LAYTIME_DAYS,
                                   hub_wri_per_day=HUB_WRI_PER_DAY,
                                   nonhub_wri_per_day=NONHUB_WRI_PER_DAY):
    """
    Calculate total voyage cost distinguishing hub vs non-hub ports.
    - route_name: "A -> B -> C"
    - route_info: dict with 'distance_km' and High risk flag
    - key_stops: set of ports considered hubs
    """
    distance_km = route_info.get("distance_km", 0.0)
    sea_days = km_to_days(distance_km)

    # ports sequence
    ports = parse_route_ports(route_name)
    num_ports = len(ports)

    # classify each stop if it is a hub (True) or non-hub (False)
    is_hub = [p in key_stops for p in ports]

    # compute aggregate port charges, laytime, demurrage and WRI contributions
    port_charge_total = 0.0
    total_laytime_days = 0.0
    # for demurrage we choose a simple rule: if any non-hub present, use non-hub demurrage expectation
    demurrage_expected = hub_demurrage  # default: hub-level
    # For WRI use max of per-port WRI rate (or you can sum/average - here we use max sensible approach)
    wri_per_day_used = 0.0

    for port, hub_flag in zip(ports, is_hub):
        if hub_flag:
            port_charge_total += hub_port_charge
            total_laytime_days += hub_laytime
            wri_per_day_used = max(wri_per_day_used, hub_wri_per_day)
        else:
            port_charge_total += nonhub_port_charge
            total_laytime_days += nonhub_laytime
            # presence of any non-hub raises demurrage expectation to nonhub level
            demurrage_expected = nonhub_demurrage
            wri_per_day_used = max(wri_per_day_used, nonhub_wri_per_day)

    total_voyage_days = sea_days + total_laytime_days

    # TCH and fuel scale with total voyage days
    TCH_total = TCH_per_day * total_voyage_days
    Fuel_total = fuel_consumption_per_day * fuel_price_per_mt * total_voyage_days

    # War risk: additionally, if route_info flagged as crossing JWC zone, add big WRI
    # Accept either key capitalization
    hr = route_info.get("High risk", route_info.get("High risk", "NIL"))
    jwc_flag = bool(hr and hr != "NIL")
    if jwc_flag:
        # If route crosses JWC zone, WRI is severe (use higher market rate override)
        # Choose a conservative large value (you may substitute a market rate lookup)
        jwc_wri_override_per_day = 35_000
        wri_total = jwc_wri_override_per_day * total_voyage_days
    else:
        wri_total = wri_per_day_used * total_voyage_days

    # total port charges already summed above (per port)
    port_charges_total = port_charge_total

    # Demurrage: use demurrage_expected (single expected total for route)
    demurrage_total = demurrage_expected

    total_cost = TCH_total + Fuel_total + port_charges_total + demurrage_total + wri_total

    breakdown = {
        "distance_km": distance_km,
        "sea_days": sea_days,
        "total_laytime_days": total_laytime_days,
        "total_voyage_days": total_voyage_days,
        "TCH": TCH_total,
        "Fuel": Fuel_total,
        "Port Charges": port_charges_total,
        "Demurrage (expected)": demurrage_total,
        "WRI": wri_total,
        "Total Cost": total_cost,
        "ports": ports,
        "is_hub_flags": is_hub,
        "jwc_crossed_flag": jwc_flag
    }

    return total_cost, breakdown


def rank_best_routes(origin, destination, routes_json="all_routes_info.json", key_hubs=set(), top_n=5):
    """
    Rank the best routes from origin to destination using calculate_cost_with_port_types().
    - origin: starting port
    - destination: ending port
    - routes_json: JSON file with precomputed routes
    - key_hubs: set of ports considered hub ports
    - top_n: number of best routes to return
    """

    with open(routes_json, "r") as f:
        all_routes = json.load(f)

    evaluated = []
    for route_name, route_info in all_routes.items():
        # Ensure origin and destination are first/last
        ports = [p.strip() for p in route_name.split("->") if p.strip()]
        if ports[0] == origin and ports[-1] == destination:
            total_cost, breakdown = calculate_cost_with_port_types(route_name, route_info, key_stops=key_hubs)
            evaluated.append({
                "route_name": route_name,
                "total_cost": total_cost,
                "breakdown": breakdown
            })

    # Sort by total cost
    ranked = sorted(evaluated, key=lambda x: x["total_cost"])[:top_n]
    return ranked

def visualize_optimized_routes(origin, destination, routes_json="all_routes_info.json",
                               key_hubs=set(), warzones=WAR_ZONES, top_n=3,
                               map_filename="optimized_routes_map.html"):
    """
    Visualize top N optimized routes between origin and destination,
    overlay ports and warzones on a fresh folium map.
    """
    # Load routes
    with open(routes_json, "r") as f:
        all_routes = json.load(f)
    
    # Filter routes where origin is first and destination is last
    filtered_routes = {
        r_name: r_info
        for r_name, r_info in all_routes.items()
        if r_name.startswith(origin) and r_name.endswith(destination)
    }
    
    if not filtered_routes:
        print(f"No routes found from {origin} to {destination}.")
        return None
    
    # Rank routes by total cost
    ranked_routes = []
    for r_name, r_info in filtered_routes.items():
        cost, breakdown = calculate_cost_with_port_types(r_name, r_info, key_stops=key_hubs)
        ranked_routes.append({
            "route_name": r_name,
            "total_cost": cost,
            "waypoints": r_info['waypoints']
        })
    
    ranked_routes = sorted(ranked_routes, key=lambda x: x['total_cost'])[:top_n]
    
    # Initialize map centered between origin and destination
    origin_coords = PORTS[origin]['coordinates']
    dest_coords = PORTS[destination]['coordinates']
    fmap = folium.Map(location=[(origin_coords[0]+dest_coords[0])/2,
                                (origin_coords[1]+dest_coords[1])/2], zoom_start=3)
    
    # --- Add ports ---
    port_colors = {'export': 'green', 'import': 'blue', 'both': 'black'}
    for port_name, info in PORTS.items():
        lat, lon = info['coordinates']
        role = info['role']
        color = port_colors.get(role, 'gray')
        popup_text = f"{port_name}<br>Role: {role}"
        folium.Marker(
            location=[lat, lon],
            popup=popup_text,
            icon=folium.Icon(color=color, icon='ship', prefix='fa')
        ).add_to(fmap)
    
    # --- Add warzones ---
    wz_colors = {
        'Gulf of Aden & Southern Red Sea':'red',
        'Sea of Azov':'darkred',
        'Black Sea (Western Maritime)':'orange',
        'Gulf of Guinea':'purple'
    }
    for wz_name, wz_info in warzones.items():
        poly_coords = [(lat, lon) for lat, lon in wz_info['bounds']]
        folium.Polygon(
            locations=poly_coords,
            color=wz_colors.get(wz_name,'gray'),
            weight=3,
            fill=True,
            fillColor=wz_colors.get(wz_name,'gray'),
            fillOpacity=0.2,
            popup=f"{wz_name}: {wz_info.get('description','')}"
        ).add_to(fmap)
    
    # --- Add top N routes ---
    colors = ["blue", "green", "red", "orange", "purple"]
    for idx, route in enumerate(ranked_routes):
        points = [(lat, lon) for lon, lat in route['waypoints']]  # convert lon/lat to lat/lon
        folium.PolyLine(points, color=colors[idx % len(colors)], weight=3.5, opacity=0.8,
                        tooltip=f"Rank {idx+1}: {route['route_name']} | Cost: ${route['total_cost']:,}").add_to(fmap)
        
        # Start marker
        folium.CircleMarker(location=points[0], radius=6, color=colors[idx % len(colors)],
                            fill=True, fill_color=colors[idx % len(colors)],
                            fill_opacity=1.0, tooltip=f"Route Rank {idx+1} Start").add_to(fmap)
        # End marker
        folium.CircleMarker(location=points[-1], radius=6, color=colors[idx % len(colors)],
                            fill=True, fill_color=colors[idx % len(colors)],
                            fill_opacity=1.0, tooltip=f"Route Rank {idx+1} End").add_to(fmap)
    
    # Save map
    fmap.save(map_filename)
    print(f"Optimized routes map saved as '{map_filename}'")
    return fmap


Map saved as 'ports_and_warzones_map.html'


Code to generate all routes (No need to run - routes saved in "all_routes_info.json")

In [4]:
# Step 1: Generate all export -> import port pairs
# export_ports = [port for port, info in PORTS.items() if info['role'] in ('export', 'both')]
# import_ports = [port for port, info in PORTS.items() if info['role'] in ('import', 'both')]

# all_port_pairs = [(origin, dest) for origin in export_ports for dest in import_ports]

# # Step 2: Calculate all routes using our calculate_routes() function
# all_routes_info = calculate_routes(all_port_pairs)

# # Step 3: Optional - print a few routes to check
# for i, (route, info) in enumerate(all_routes_info.items()):
#     if i < 5:  # only show first 5 for brevity
#         print(f"{route}: Distance = {info['distance_km']} km, War zone crossed = {info['war_zone_crossed']}")

# # Step 4: Save all routes to a JSON file for future optimization
# import json
# with open("all_routes_info.json", "w") as f:
#     json.dump(all_routes_info, f, indent=4)

# print("All routes calculated and saved to 'all_routes_info.json'.")

Route generation sample testing

In [5]:
with open("all_routes_info.json", "r") as f:
    all_routes_info = json.load(f)

# Check how many routes were generated
print(f"Total routes generated: {len(all_routes_info)}")

# Check some example routes
for i, (route_name, route_data) in enumerate(all_routes_info.items()):
    print(f"{route_name}: {len(route_data['waypoints'])} waypoints, "
          f"Distance: {route_data['distance_km']:.1f} km, "
          f"War zone crossed: {route_data['war_zone_crossed']}, "
          f"High risk: {route_data['High risk']}")
    if i >= 5:
        break  # just preview first 5


Total routes generated: 3789
Rosario -> Santos: 9 waypoints, Distance: 2082.6 km, War zone crossed: None, High risk: NIL
Rosario -> Cartagena -> Santos: 96 waypoints, Distance: 19240.4 km, War zone crossed: None, High risk: NIL
Rosario -> Alexandria -> Santos: 138 waypoints, Distance: 25031.8 km, War zone crossed: None, High risk: NIL
Rosario -> Genoa -> Santos: 156 waypoints, Distance: 21546.1 km, War zone crossed: None, High risk: NIL
Rosario -> Rotterdam -> Santos: 184 waypoints, Distance: 22156.6 km, War zone crossed: None, High risk: NIL
Rosario -> Bejaia: 62 waypoints, Distance: 11152.7 km, War zone crossed: None, High risk: NIL


Route Map generation test

In [ ]:
# # Sample port pairs for testing
# sample_port_pairs = [
#     #("Santos", "Rotterdam")     # Direct + possible hub-based alternatives
#     #("Kwinana", "Jeddah")        # Likely crosses war zone → detour to Cape Town
#     ("New Orleans", "Alexandria") # Direct + alternatives based on cargo
# ]

# # Call the function to visualize these routes
# visualize_routes_for_ports(
#     port_pairs=sample_port_pairs,
#     map_filename="sample_routes_map.html",
#     routes_json="all_routes.json"  # Make sure this is your saved routes JSON
# )


Multiple Port Route generation test

In [8]:
# Example usage with your existing JSON
routes_santos_rotterdam = get_routes_from_json("Santos", "Rotterdam")
routes_neworleans_genoa = get_routes_from_json("New Orleans", "Genoa")

print("Santos -> Rotterdam Routes:")
for r_name, r_info in routes_santos_rotterdam.items():
    print(f"{r_name}: Distance {r_info['distance_km']} km, High Risk: {r_info['High risk']}")


print("\nNew Orleans -> Genoa Routes:")
for r_name, r_info in routes_neworleans_genoa.items():
    print(f"{r_name}: Distance {r_info['distance_km']} km, High Risk: {r_info['High risk']}")


Santos -> Rotterdam Routes:
Santos -> Rotterdam: Distance 10114.525231812377 km, High Risk: NIL
Santos -> Cartagena -> Rotterdam: Distance 11692.857877131299 km, High Risk: NIL
Santos -> Alexandria -> Rotterdam: Distance 17484.232403758848 km, High Risk: NIL
Santos -> Genoa -> Rotterdam: Distance 13998.528621047946 km, High Risk: NIL
Santos -> Jeddah -> Rotterdam: Distance 20651.61971159984 km, High Risk: NIL

New Orleans -> Genoa Routes:
New Orleans -> Genoa: Distance 10007.455182997772 km, High Risk: NIL
New Orleans -> Cartagena -> Genoa: Distance 10018.939417075246 km, High Risk: NIL
New Orleans -> Alexandria -> Genoa: Distance 14220.552849632855 km, High Risk: NIL
New Orleans -> Rotterdam -> Genoa: Distance 13098.73219489372 km, High Risk: NIL
New Orleans -> Jeddah -> Genoa: Distance 17362.23843779965 km, High Risk: NIL


Best Routes with Distance and Cost

In [15]:
best_routes_santos_rotterdam = rank_best_routes(
    origin="Santos",
    destination="Rotterdam",
    routes_json="all_routes_info.json",
    key_hubs=key_hubs,
    top_n=3
)

best_routes_neworleans_genoa = rank_best_routes(
    origin="New Orleans",
    destination="Genoa",
    routes_json="all_routes_info.json",
    key_hubs=key_hubs,
    top_n=3
)

# Display results
for r in best_routes_santos_rotterdam:
    print(r['route_name'], r['total_cost'])
for r in best_routes_neworleans_genoa:
    print(r['route_name'], r['total_cost'])

Santos -> Rotterdam 1875938.5306948454
Santos -> Cartagena -> Rotterdam 2374199.7668761667
Santos -> Genoa -> Rotterdam 2549902.5249570822
New Orleans -> Genoa 1867779.2985346508
New Orleans -> Cartagena -> Genoa 2246639.450204586
New Orleans -> Rotterdam -> Genoa 2481333.8820995647


Map Visuals for N best Routes example

In [17]:
# Visualize top 3 optimized routes for Santos -> Rotterdam
fmap1 = visualize_optimized_routes("Santos", "Rotterdam",
                                   routes_json="all_routes_info.json",
                                   key_hubs=key_hubs,
                                   top_n=3,
                                   map_filename="santos_rotterdam_optimized_map.html")

# Visualize top 3 optimized routes for New Orleans -> Genoa
fmap2 = visualize_optimized_routes("New Orleans", "Genoa",
                                   routes_json="all_routes_info.json",
                                   key_hubs=key_hubs,
                                   top_n=3,
                                   map_filename="neworleans_genoa_optimized_map.html")

Optimized routes map saved as 'santos_rotterdam_optimized_map.html'
Optimized routes map saved as 'neworleans_genoa_optimized_map.html'
